https://developers.google.com/protocol-buffers/docs/pythontutorial

create an address book with a contact's name, id, email, and phone number.

1. define protocol format by writing the following to a `.proto` file:

```proto
syntax = "proto2";

package tutorial;

message Person {
  required string name = 1;
  required int32 id = 2;
  optional string email = 3;
  
  enum PhoneType {
    MOBILE = 0;
    HOME = 1;
    WORK = 2;
  }
  
  message PhoneNumber {
    required string number = 1;
    optional PhoneType type = 2 [default = HOME];
  }
  
  repeated PhoneNumber phones = 4;
}

message AddressBook {
  repeated Person people = 1;
}
```

Explanation:

- `package` prevents name space conflict. no effect on python, but should still be declared in case the file gets used in other languages
- a `message` is a collection of typed data
- an `enum` is a predefined list of values
- `= 1`, `= 2` are field IDs. they're used to identify your fields in the message binary format in order to mark which field is associated with a value. This means that renaming a field isn't a breaking change (in terms of wire format) and the names themselves don't have to be serialized.
- each field must have these rules: `required`, `optional`, `repeated`
- `required` fields must be set. otherwise exceptions will occur
- `optional` fields can remain unset. if unset, default values are used: 0 for int; '' for string; bool for boolean.
- `repeated` fields can be repeated any number of times (include 0). the order is preserved.
- note: **`require` is forever**! You cannot change protocol fields marked as `required` to `optional`, and expect the compile protocol buffers to interoperate. This is why these keywords are removed in protobuf v3.

Note on protobuff v3:

- `required` and `optional` are deprecated

2. Compiling the `.proto` file

In [7]:
# !protoc ./my_protobuf.proto --python_out='.'

In [11]:
# !ls

my_protobuf.proto   protobuf-3.11.2	      protoc-3.11.2-binary
my_protobuf_pb2.py  protobuff-tutorial.ipynb


Now we have an extra file created by the protobuf compiler: `my_protobuf_pb2.py`

Take a moment to open and read the file in a text editor. It's ok if you don't understand most of the code in it. Note how the structure resembles a dict.

When done, come back and run the code below.

3. Import the `.py` file into python. Protobuf binary files can only be created or read using this script:

### The Protocol Buffer API

In [1]:
import my_protobuf_pb2

In [2]:
dir(my_protobuf_pb2)

['AddressBook',
 'DESCRIPTOR',
 'Person',
 '_ADDRESSBOOK',
 '_PERSON',
 '_PERSON_PHONENUMBER',
 '_PERSON_PHONETYPE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_descriptor',
 '_message',
 '_reflection',
 '_sym_db',
 '_symbol_database']

#### Create a `Person` message (object)

In [3]:
person = my_protobuf_pb2.Person()

In [4]:
type(person)

my_protobuf_pb2.Person

In [5]:
dir(person)

['ByteSize',
 'Clear',
 'ClearExtension',
 'ClearField',
 'CopyFrom',
 'DESCRIPTOR',
 'DiscardUnknownFields',
 'Extensions',
 'FindInitializationErrors',
 'FromString',
 'HOME',
 'HasExtension',
 'HasField',
 'IsInitialized',
 'ListFields',
 'MOBILE',
 'MergeFrom',
 'MergeFromString',
 'ParseFromString',
 'PhoneNumber',
 'PhoneType',
 'RegisterExtension',
 'SerializePartialToString',
 'SerializeToString',
 'SetInParent',
 'UnknownFields',
 'WORK',
 'WhichOneof',
 '_CheckCalledFromGeneratedFile',
 '_SetListener',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_extensions_by_name',
 '_extensions_by_number',
 'email',


Recall that `Person` has these fields:

```proto
message Person {
  required string name = 1;
  required int32 id = 2;
  optional string email = 3;

  ...

  repeated PhoneNumber phones = 4;
```

#### Default values:

In [6]:
person.IsInitialized()

False

In [7]:
person.name

''

In [8]:
person.id

0

In [9]:
person.email

''

Assign values to person object's attributes:

In [10]:
person.id = 42
person.id

42

In [11]:
person.name = 'Harry Potter'
person.name

'Harry Potter'

In [12]:
person

name: "Harry Potter"
id: 42

Once all `required` fields are assigned a value, the object is considered initialized.

In [13]:
person.IsInitialized()

True

In [14]:
person.email = 'hpotter@hogwarts.edu.uk'
person.email

'hpotter@hogwarts.edu.uk'

In [15]:
person

name: "Harry Potter"
id: 42
email: "hpotter@hogwarts.edu.uk"

Access non-existent attribute:

In [16]:
person.xxx

AttributeError: xxx

#### Type checking is built-in before an attribute can be updated:

In [17]:
person.id = 'xxx'

TypeError: 'xxx' has type str, but expected one of: int, long

In [18]:
person

name: "Harry Potter"
id: 42
email: "hpotter@hogwarts.edu.uk"

#### Add a `PhoneNumber` message object:

from `.proto`:

```proto
repeated PhoneNumber phones = 4;
```

In [19]:
person.phones.add()

In [20]:
person

name: "Harry Potter"
id: 42
email: "hpotter@hogwarts.edu.uk"
phones {
}

In [21]:
person.phones.add()

In [22]:
person

name: "Harry Potter"
id: 42
email: "hpotter@hogwarts.edu.uk"
phones {
}
phones {
}

In [23]:
person.phones.pop()

In [24]:
person.phones

[]

In [25]:
person.phones.pop()

In [26]:
person

name: "Harry Potter"
id: 42
email: "hpotter@hogwarts.edu.uk"

In [27]:
phone_num = person.phones.add()

In [28]:
person

name: "Harry Potter"
id: 42
email: "hpotter@hogwarts.edu.uk"
phones {
}

In [29]:
type(phone_num)

my_protobuf_pb2.PhoneNumber

From `.proto`:

```proto
message PhoneNumber {
required string number = 1;
optional PhoneType type = 2 [default = HOME];
}
```

In [30]:
phone_num.number = "(02)2222-3333"

In [31]:
phone_num

number: "(02)2222-3333"

In [32]:
person.PhoneType.items()

[('MOBILE', 0), ('HOME', 1), ('WORK', 2)]

In [33]:
phone_num.type = 0

In [34]:
phone_num

number: "(02)2222-3333"
type: MOBILE

#### Create an `AddressBook` message object

```proto
message AddressBook {
  repeated Person people = 1;
}
```

In [35]:
# create an AddressBook message
addr_bk = my_protobuf_pb2.AddressBook()

In [36]:
addr_bk

In [37]:
type(addr_bk)

my_protobuf_pb2.AddressBook

In [38]:
dir(addr_bk)

['ByteSize',
 'Clear',
 'ClearExtension',
 'ClearField',
 'CopyFrom',
 'DESCRIPTOR',
 'DiscardUnknownFields',
 'Extensions',
 'FindInitializationErrors',
 'FromString',
 'HasExtension',
 'HasField',
 'IsInitialized',
 'ListFields',
 'MergeFrom',
 'MergeFromString',
 'ParseFromString',
 'RegisterExtension',
 'SerializePartialToString',
 'SerializeToString',
 'SetInParent',
 'UnknownFields',
 'WhichOneof',
 '_CheckCalledFromGeneratedFile',
 '_SetListener',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_extensions_by_name',
 '_extensions_by_number',
 'people']

In [39]:
addr_bk.IsInitialized()

True

In [40]:
addr_bk.people

[]

In [41]:
type(addr_bk.people)

google.protobuf.pyext._message.RepeatedCompositeContainer

In [42]:
dir(addr_bk.people)

['MergeFrom',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add',
 'append',
 'extend',
 'insert',
 'pop',
 'remove',
 'sort']

#### Add `person` to an `AddressBook` message object

`AddressBook` message has field `people`:

```proto
repeated Person people = 1;
```

In [43]:
addr_bk.people.add(person)

TypeError: No positional arguments allowed

Turns out, `addr_bk.people.add()` doesn't take an argument. Instead, it **returns** a `person` message (object), and you're suppose to modify that object.

So instead of creating a person, and adding it to an address book, you'd have to do it in reverse order:

1. create an `AddressBook` message object
1. call `AddressBook.people.add()` to create a `Person` message object and assign it to a variable, and *then* modify that variable.

Google's tutorial on [writing a message](https://developers.google.com/protocol-buffers/docs/pythontutorial#writing-a-message) starts off with a function that modifies a `Person` object. But the meat of the process is actually at the bottom of the code window:

```python
# Create an AddressBook message object
address_book = addressbook_pb2.AddressBook()

# Read an existing address book from binary file.
...
f = open("my_addr_book_pb2", "rb")
address_book.ParseFromString(f.read())
f.close()

# Add an address
PromptForAddress(address_book.people.add())

# Write the updated address book to file
f = open("my_addr_book_pb2", "wb")
f.write(address_book.SerializeToString())
f.close()
```

So in essence:
1. start by creating an `Addressbook` message object
1. load an address book by calling `address_book.ParseFromString(file_object)`
1. create a `Person` object by calling `address_book.people.add()` and modify that object directly with a function called `PromptForAddress(person_object)`.
1. export the updated address book with `address_book.SerializeToString()`

See [`protobuff-tutorial-end-to-end.ipynb`](./protobuff-tutorial-end-to-end.ipynb) for an end-to-end tutorial in the correct order.